In [41]:
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt

In [42]:
url = 'https://quke.ru/shop/smartfony?pfrom=379&pto=148490&availability=instock&page-size=72'
response = requests.get(url)
print(response)

<Response [200]>


In [43]:
bs = BeautifulSoup(response.text)

In [63]:
# url = 'https://quke.ru/shop/smartfony?pfrom=379&pto=148490&availability=instock&page-size=72'
url = 'https://quke.ru/shop/smartfony?availability=instock&pto=193990&page-size=72'

max_pages = 11

price_list = []
names_list = []
articul_list = []
screen = []
inch = []
main_camera = []
second_camera = []
processor_name = []
processor_rate = []
internal_memory = []
ram = []
battery = []


for p in range(max_pages):
    cur_url = url + '&page=' + str(p + 1)
    html_text = requests.get(cur_url).text
    soup = BeautifulSoup(html_text, 'lxml')
    p = soup.find_all('span', 'b-card2-v2__price-val') 
    for i in p:
        price = i.text
        if len(price.split()) != 1:
            price_list.append(int(str(price.split()[0]) + str(price.split()[1])))
        else:
            price_list.append(int(price))
    
    p = soup.find_all('a', 'b-card2-v2__name') 
    for i in p:
        name = i.text
        names_list.append(name)
        
    p = soup.find_all('div', 'b-card2-v2__artnumber') 
    for i in p:
        articul = i.text
        articul_list.append(int(articul.split(':')[1]))
    
    p = soup.find_all('div', 'b-card2-v2') 
    for i in p:
        info_for_one = i.find_all('span', 'value')
        if len(info_for_one) == 0:
            screen.append(None)
        else:
            screen.append(info_for_one[0].text.split(',')[0])
        
    p = soup.find_all('div', 'b-card2-v2') 
    for j in p:
        inch_for_one = j.find_all('span', 'value')
        if len(inch_for_one) != 0:
            if len(inch_for_one[0].text.split(',')) == 1:
                inch.append(None)
            else:
                if float(inch_for_one[0].text.split(',')[1].split()[0]) > 10:
                    inch.append(None)
                else:
                    inch.append(float(inch_for_one[0].text.split(',')[1].split()[0]))
        else:
            inch.append(None)

    for j in p:
        main_camera_for_one = j.find_all('span', 'value')
        if len(main_camera_for_one) != 0 and len(main_camera_for_one) != 1:
            photo = main_camera_for_one[1].text.split()
            if len(photo) == 1:
                main_camera.append(None)
            elif photo[1] == 'Мп' or photo[1] == 'Мп,':
                main_camera.append(float(photo[0]))
            elif photo[1] == 'мАч':
                main_camera.append(None)
            else:
                if photo[0].isdigit():
                    main_camera.append(float(photo[0]))
                else:
                    main_camera.append(None)
            if len(photo) == 6:
                second_camera.append(float(photo[4]))
            else:
                second_camera.append(None)
        else:
            main_camera.append(None)
            second_camera.append(None)
    
    for j in p:
        proc = j.find_all('span', 'value')
        if len(proc) >=4:
            if len(proc[3].text.split()) != 1:
                if len(proc[3].text.split(', ')) != 1:
                    processor_name.append(proc[3].text.split(', ')[0])
                    processor_rate.append(float(proc[3].text.split(', ')[1].split()[0]))
                else:
                    processor_name.append(None)
                    processor_rate.append(None)
            else:
                processor_name.append(None)
                processor_rate.append(None)
        else:
            processor_name.append(None)
            processor_rate.append(None)
    
    for j in p:
        memory = j.find_all('span', 'value')
        if len(memory) > 3:
            if len(memory[2].text) != 1:
                if '/' in memory[2].text:
                    memory = memory[2].text.split('/')
                    internal_memory.append(float(memory[0].split()[0]))
                    ram.append(float(memory[1].split()[0]))
                else:
                    if 'ГБ' in memory[2].text:
                        internal_memory.append(float(memory[2].text.split()[0]))
                        ram.append(None)
                    else:
                        internal_memory.append(None)
                        ram.append(None)
            else:
                internal_memory.append(None)
                ram.append(None)
        else:
            internal_memory.append(None)
            ram.append(None)

    for j in p:
        bat = j.find_all('span', 'value')
        if len(bat) > 4:
            battery.append(int(bat[4].text.split()[0]))
        else:
            battery.append(None)  

In [65]:
import pandas as pd
df = pd.DataFrame({'Name':names_list,
             'Price': price_list,
             'Articul': articul_list,
              'Screen': screen, 
              'Inch': inch, 
              'Main camera': main_camera,
              'Second camera': second_camera,
              'Processor name': processor_name, 
              'Processor rate': processor_rate, 
              'Internal memory': internal_memory,
              'RAM': ram, 
              'Battery': battery
             })

In [66]:
df

,Name,Price,Articul,Screen,Inch,Main camera,Second camera,Processor name,Processor rate,Internal memory,RAM,Battery
0,Xiaomi Redmi Note 11 NFC 4 64Gb RU Star Blue,11790,111788,2400x1080,6.43,50.0,13.0,Snapdragon 680 4G,2.4,64.0,4.0,5000.0
1,Xiaomi Redmi 10C 4 64Gb RU Green,8350,114089,1650x720,6.71,50.0,5.0,Snapdragon 680 4G,2.4,64.0,4.0,5000.0
2,Xiaomi Redmi Note 12 6 128Gb NFC RU Onyx Gray,15350,126457,2400x1080,6.67,50.0,13.0,Snapdragon 685,2.8,128.0,6.0,5000.0
3,Realme C55 8 256Gb Black,14850,126955,2400x1080,6.72,64.0,8.0,Helio G88,2.0,256.0,8.0,5000.0
4,Xiaomi Redmi 10C 4 64Gb RU Blue,8350,112138,1650x720,6.71,50.0,5.0,Snapdragon 680 4G,2.4,64.0,4.0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
514,Xiaomi Redmi Note 12 Pro 5G 6 128Gb EU Blue,26990,118992,2400x1080,6.67,50.0,16.0,Dimensity 1080,2.6,128.0,6.0,5000.0
515,Xiaomi Redmi Note 12 Pro 5G 8 128Gb EU Blue,28990,118995,2400x1080,6.67,50.0,16.0,Dimensity 1080,2.6,128.0,8.0,5000.0
516,Xiaomi Redmi Note 12 Pro 5G 8 128Gb EU Pink,28990,118994,2400x1080,6.67,50.0,16.0,Dimensity 1080,2.6,128.0,8.0,5000.0
517,Xiaomi Redmi Note 12 Pro 5G 8 128Gb EU White,28990,118996,2400x1080,6.67,50.0,16.0,Dimensity 1080,2.6,128.0,8.0,5000.0


In [67]:
df.to_csv('project_data')